In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122600")
exp = Experiment(workspace=ws, name="quick-starts-ws-122600")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122600
Azure region: southcentralus
Subscription id: 2248b1c7-a065-4265-9476-038b58f0650e
Resource group: aml-quickstarts-122600


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_cluster = ComputeTarget(workspace=ws, name="whitehat")
    print('Found existing compute target {}.'.format("whitehat"))
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, "whitehat", config)
    compute_cluster.wait_for_completion(show_output=True)

Found existing compute target whitehat.


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--max_iter': [200,300,400,50],
    '--C': [0.01, 0.1, 1.0,0.001]
}

)

# Specify a Policy
policy = BanditPolicy(slack_amount = 0.2, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="training",
                          pip_packages=['azureml-sdk'],
                          entry_script='train.py',
                          compute_target = compute_cluster,)
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name='Accuracy', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6,
                          max_concurrent_runs=2)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

experiment = Experiment(workspace = ws, name = 'Banking_Data_training_hyperdrive')
run = experiment.submit(config=hyperdrive_config)


from azureml.widgets import RunDetails
RunDetails(run).show()
run.wait_for_completion()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_e527e4d1-aba8-470a-9fe6-dce9ee14883e',
 'target': 'whitehat',
 'status': 'Canceled',
 'startTimeUtc': '2020-10-24T11:01:46.367138Z',
 'endTimeUtc': '2020-10-24T11:05:47.852248Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'details': [],
   'debugInfo': {'message': '[(https://southcentralus.experiments.azureml.net/subscriptions/2248b1c7-a065-4265-9476-038b58f0650e/resourceGroups/aml-quickstarts-122600/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-122600/experiments/**SCRUBBED**/runs/HD_e527e4d1-aba8-470a-9fe6-dce9ee14883e_0, {\'error\': {\n    "additional_properties": {\n        "componentName": "execution-worker"\n    },\n    "error": {\n        "additional_properties": {\n            "severity": null,\n            "referenceCode": null,\n            "messageParameters": null,\n            "messageFormat": null\n        },\n        "code": "UserError",\n        "message":

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run_hyperdrive = run.get_best_run_by_primary_metric()
best_run_hyperdrive.register_model(model_path='outputs/model.pkl', model_name='Best_model_Hyper_drive')

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory as tdf

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
### YOUR CODE HERE ###
ds = tdf.from_delimited_files(data_url)

In [15]:
# Use the clean_data function to clean your data.

train_ds, test_ds = ds.random_split(percentage=0.7, seed=111)



In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    compute_target=compute_cluster,
    training_data=train_ds,
    validation_data=test_ds,
    label_column_name='y',
    n_cross_validations= 3,
    featurization='auto')

In [17]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.widgets import RunDetails

automl_experiment = Experiment(ws, 'Banking_data_automl')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
print("Model Trained using AutoML !!")

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_automl, fitted_model = automl_run.get_output()

best_run_automl.register_model(model_path='outputs/model.pkl', model_name='Best_model_automl')

In [ ]:
## Deleting the compute cluster created !!
from azureml.core.compute import delete_compute
delete_compute(compute_cluster)